<a href="https://colab.research.google.com/github/isazanlorenzi/Atividade01/blob/main/DSPy_txt2sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
import threading
import subprocess

# Start Ollama service in background

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()

In [ ]:
!ollama pull gemma3:1b

In [ ]:
### https://www.codecademy.com/article/what-is-dspy

In [ ]:
!pip install dspy-ai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.2/285.2 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 102.4 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.76.0
    Uninstalling grpcio-1.76.0:
      Successfully uninstalled grpcio-1.76.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires grpcio>=1.71.2, but you have grpcio 1.67.1 which is incompatible.


In [ ]:
import dspy
import requests

In [ ]:
def setup_ollama():
  """Configure DSPy to use Ollama."""
  try:
    requests.get("http://localhost:11434/api/tags")
    print("Ollama is running")
  except:
    print("Ollama not running. Start with: ollama serve")
    return False

In [ ]:
dspy.settings.configure(
    lm=dspy.LM(
      model="ollama/gemma3:1b",
      api_base="http://localhost:11434",
      max_tokens=500,
      temperature=0.7,
    )
  )

In [ ]:
import sqlite3

def create_sample_database():
  """Create and populate sample database."""
  conn = sqlite3.connect("company.db")
  c = conn.cursor()

  # Create tables
  c.execute("""
    CREATE TABLE IF NOT EXISTS employees (
      id INTEGER PRIMARY KEY, name TEXT, department TEXT,
      salary INTEGER, hire_date TEXT
    )
  """)

  c.execute("""
    CREATE TABLE IF NOT EXISTS departments (
      id INTEGER PRIMARY KEY, name TEXT,
      budget INTEGER, manager TEXT
    )
  """)

  c.execute("""
    CREATE TABLE IF NOT EXISTS projects (
      id INTEGER PRIMARY KEY, name TEXT, department_id INTEGER,
      budget INTEGER, status TEXT
    )
  """)

  # Insert data
  c.executemany("INSERT OR IGNORE INTO employees VALUES (?, ?, ?, ?, ?)", [
    (1, "Alice Johnson", "Engineering", 95000, "2022-01-15"),
    (2, "Bob Smith", "Sales", 75000, "2021-06-20"),
    (3, "Carol White", "Engineering", 90000, "2023-02-10"),
    (4, "David Brown", "Marketing", 70000, "2022-09-05"),
    (5, "Eve Davis", "Engineering", 98000, "2020-03-12"),
  ])

  c.executemany("INSERT OR IGNORE INTO departments VALUES (?, ?, ?, ?)", [
    (1, "Engineering", 500000, "Alice Johnson"),
    (2, "Sales", 300000, "Bob Smith"),
    (3, "Marketing", 200000, "Carol White"),
  ])

  c.executemany("INSERT OR IGNORE INTO projects VALUES (?, ?, ?, ?, ?)", [
    (1, "API Development", 1, 150000, "In Progress"),
    (2, "Mobile App", 1, 200000, "Planning"),
    (3, "Sales Dashboard", 2, 80000, "Complete"),
  ])

  conn.commit()
  conn.close()
  print("Database created")


In [ ]:
### signatures to declare the input-output behavior

In [ ]:
class GenerateSQL(dspy.Signature):
  """Generate SQL from natural language.

  Database schema:
  - employees: id, name, department, salary, hire_date
  - departments: id, name, budget, manager
  - projects: id, name, department_id, budget, status
  """

  question = dspy.InputField(desc="Natural language question")
  sql_query = dspy.OutputField(desc="Valid SQL query")


In [ ]:
### SQL generator module

In [ ]:
class SQLGenerator(dspy.Module):
  """Generate and execute SQL queries with error refinement."""
  def __init__(self):
    super().__init__()
    self.generator = dspy.ChainOfThought(GenerateSQL)
    self.refiner = dspy.ChainOfThought(
      "question, sql_query, error -> refined_sql"
    )

  def forward(self, question, conn):
    # Generate SQL
    output = self.generator(question=question)
    sql = output.sql_query.strip()

    # Try to execute
    try:
      results = conn.execute(sql).fetchall()
      return dspy.Prediction(
        sql_query=sql, results=results, error=None
      )
    except Exception as e:
      # Refine on error
      refined = self.refiner(
        question=question, sql_query=sql, error=str(e)
      )
      try:
        results = conn.execute(refined.refined_sql).fetchall()
        return dspy.Prediction(sql_query=refined.refined_sql, results=results, error=None)
      except Exception as e2:
        return dspy.Prediction(
          sql_query=refined.refined_sql, results=None, error=str(e2)
        )


In [ ]:
### Training examples

In [ ]:
def create_examples():
  """Training examples for few-shot learning."""
  return [
    dspy.Example(
      question="How many employees are in the Engineering department?",
      sql_query="SELECT COUNT(*) FROM employees WHERE department = 'Engineering'"
    ).with_inputs("question"),
      dspy.Example(
        question="What is the highest salary in Engineering?",
        sql_query="SELECT MAX(salary) FROM employees WHERE department = 'Engineering'"
      ).with_inputs("question"),
        dspy.Example(
          question="List all employees with salary above 90000",
          sql_query="SELECT name, salary FROM employees WHERE salary > 90000"
        ).with_inputs("question"),
        dspy.Example(
          question="Which department has the highest budget?",
          sql_query="SELECT name, budget FROM departments ORDER BY budget DESC LIMIT 1"
        ).with_inputs("question"),
        dspy.Example(
          question="Show all projects in progress",
          sql_query="SELECT name, budget, status FROM projects WHERE status = 'In Progress'"
        ).with_inputs("question"),
    ]


In [ ]:
setup_ollama()

Ollama is running


In [ ]:
create_sample_database()

Database created


In [ ]:
# Setup generator with few-shot examples
generator = SQLGenerator()
generator.generator.demos = create_examples()
print("✓ Generator configured with few-shot examples")

✓ Generator configured with few-shot examples


In [ ]:
conn = sqlite3.connect("company.db")

In [ ]:
test_questions = [
    "How many employees work in Engineering?",
    "What is the average salary of all employees?",
    "Show me all projects with a budget greater than 100000",
    "Which employees have been hired since 2022?",
    "List all managers and their departments",
]

In [ ]:
for i, question in enumerate(test_questions, 1):
    print(f"Query {i}: {question}")
    result = generator.forward(question, conn)
    print(f"SQL: {result.sql_query}")
    print(f"Results: {result.results if not result.error else f' {result.error}'}\n")

Query 1: How many employees work in Engineering?


2025/12/16 17:28:00 WARNING dspy.primitives.module: Calling module.forward(...) on SQLGenerator directly is discouraged. Please use module(...) instead.
2025/12/16 17:28:49 WARNING dspy.primitives.module: Calling module.forward(...) on SQLGenerator directly is discouraged. Please use module(...) instead.


SQL: SELECT COUNT(*) FROM employees WHERE department = 'Engineering';
Results: [(3,)]

Query 2: What is the average salary of all employees?


2025/12/16 17:29:30 WARNING dspy.primitives.module: Calling module.forward(...) on SQLGenerator directly is discouraged. Please use module(...) instead.


SQL: SELECT AVG(salary) FROM employees;
Results: [(85600.0,)]

Query 3: Show me all projects with a budget greater than 100000


2025/12/16 17:30:36 WARNING dspy.primitives.module: Calling module.forward(...) on SQLGenerator directly is discouraged. Please use module(...) instead.


SQL: SELECT * FROM projects WHERE budget > 100000;
Results: [(1, 'API Development', 1, 150000, 'In Progress'), (2, 'Mobile App', 1, 200000, 'Planning')]

Query 4: Which employees have been hired since 2022?


2025/12/16 17:32:50 WARNING dspy.primitives.module: Calling module.forward(...) on SQLGenerator directly is discouraged. Please use module(...) instead.


SQL: SELECT e.id, e.name, e.department, e.salary, e.hire_date FROM employees e JOIN (SELECT hire_date FROM hire_dates WHERE hire_date > '2022-01-01') AS hire_dates ON e.id = hire_dates.id WHERE hire_date > '2022-01-01';
Results:  no such table: hire_dates

Query 5: List all managers and their departments
SQL: SELECT e.name, d.name
FROM employees e
JOIN departments d ON e.department = d.department_id
GROUP BY e.name, d.name;
Results:  no such column: d.department_id



In [ ]:
### https://dspy.ai/tutorials/gepa_facilitysupportanalyzer/    estudar!!!!!!

In [ ]:
https://dspy.ai/tutorials/mcp/